In [2]:
# Importar librerías 

import pandas as pd  

from bs4 import BeautifulSoup
import requests

import re

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [1]:
# Fase 1
class ExtraerDatosPeliculasApi:
    def __init__(self, key, anho_inicial, anho_final, genero, pags): 
        self.key = key 
        self.anho_inicial = anho_inicial
        self.anho_final = anho_final
        self.genero = genero
        self.pags = pags
        self.peliculas = []

        self.llamar_api()

    def llamar_api(self):
        """
        Realiza una llamada a una API utilizando la URL proporcionada.
        Parameters:
        url (str): La URL de la API que se va a llamar.
        Returns:
        dict or None: Un diccionario con los datos de respuesta de la API si la llamada fue exitosa (código de estado 200).
                    None si la llamada falló o no se pudo autenticar correctamente.
        """
        headers = {
        "X-RapidAPI-Key": self.key,
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"  
        }               
        

        # Hace la llamada a la Api cambiando los paramentros año, genero, paginas
        for anho in range(self.anho_inicial, self.anho_final+1):                
            for pag in range(1, self.pags+1):
                url = f"https://moviesdatabase.p.rapidapi.com/titles?year={anho}&genre={self.genero.capitalize()}&page={pag}"
                # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'
                
                llamada = requests.get(url, headers= headers)                
                if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
                    print(f"El motivo por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
                else: 
                    # Convierte la consulta a formato json
                    datos_en_bruto = llamada.json()
                    # Llamamos a la función limpiar datos para coger los datos que nos interesan 
                    self.limpiar_datos(datos_en_bruto)           
            print(f"La llamada del año {anho} a la API nos ha dado una respuesta de tipo: {llamada.status_code}")  # Imprime el código de estado de la respuesta.  
        print(f"Todos los datos guardados correctamente")     
      
    # Cogemos los datos que nos interesan de la llamda.json
    def limpiar_datos (self, datos_en_bruto):
        # Creamos un dic para guardar los datos
        info_peliculas = {"tipo": [],
                        "titulo": [],
                        "año": [],
                        "mes": [],
                        "id":[],
                        "genero":[]}
        for pelicula in datos_en_bruto["results"]: 
            # Sacamos el valor de la clave tipo
            tipo = pelicula["titleType"]["text"]
            if tipo == "Movie" or tipo =="Short":
                # Lo añadimos al dic dentro de la clave tipo
                info_peliculas["tipo"].append(tipo)
                # Sacamos y añadimos al dic la clave id
                info_peliculas["id"].append(pelicula["id"])
                # Sacamos el valor y añadimos al dic la clave titulo    
                info_peliculas["titulo"].append(pelicula["titleText"]["text"])
                # Como hemos detectado que algunas peliculas no tienen año tratamos esta excepcion
                # Sacamos el valor y lo añadimos al dic la clave año, si no la encuentra añadimos none
                try:
                    # Sacamos y añadimos al dic dentro de la clave año   
                    info_peliculas["año"].append(pelicula["releaseDate"]["year"])
                except:
                    # En caso de que no lo encuentre añadimos al dic dentro de la clave año el valor none
                    info_peliculas["año"].append("")                
                try:
                    # Sacamos y añadimos al dic dentro de la clave mes   
                    info_peliculas["mes"].append(pelicula["releaseDate"]["month"])
                except:
                    # En caso de que no lo encuentre añadimos al dic dentro de la clave mes el valor none
                    info_peliculas["mes"].append("")           
                info_peliculas["genero"].append(self.genero.capitalize())
        
    # Añade a la lista peliculas las tuplas
        self.peliculas.extend(list(zip( info_peliculas["tipo"], 
                                info_peliculas["titulo"],
                                info_peliculas["mes"],
                                info_peliculas["año"],
                                info_peliculas["id"],
                                info_peliculas["genero"]))) # Mostrar el resultado print(lista_de_tuplas)
        
        self.guardar_csv(self.genero)
        
    
    def guardar_csv (self,nombre_archivo):
        df = pd.DataFrame(self.peliculas, columns=["Tipo", "Titulo", "Mes", "Año", "Id", "Genero"])
        df.to_csv(nombre_archivo+".csv", index=False, header=True)



In [3]:
key = input("Introduce tu key")
anho_inicial = int(input("Introduce el desde que año quieres buscar"))
anho_final = int(input("Introduce hasta que año quieres buscar"))
genero = input("Introduce el género (en inglés) que quieres buscar")
numero_pag = int(input("Introduce el numero de paginas que quieres buscar"))
peliculas_api = ExtraerDatosPeliculasApi(key, anho_inicial, anho_final, genero, numero_pag)

TypeError: can only concatenate str (not "int") to str